## Producer

### Instalação dos pacotes e bibliotecas necessários

In [19]:
!pip install --upgrade pip
!pip install requests
!pip install pika
!pip install pysqlite3

In [20]:
import requests
import json
from kafka import KafkaProducer
import requests
import json

### Conexão com a API "the movie" e envio dos filmes para o tópico "atividade_kafka" 

In [26]:
# URL do endpoint da API de filmes
url = "https://api.themoviedb.org/3/movie/now_playing?api_key=faa82585bb153b3362a84dc30fe51c35&page=1&language=pt-BR®ion=br"

# Lista de títulos dos filmes desejados
filmes_desejados = [
    "Five Nights at Freddy's",
    "Retribution",
    "Muzzle",
    "Desperation Road",
    "Coco",
    "Sound of Freedom",
    "Mavka: The Forest Song",
    "Blue Beetle",
    "The Burial",
    "57 Seconds",
    "Killers of the Flower Moon",
    "Scarygirl",
    "The Nightmare Before Christmas"
]

# Configurando o produtor Kafka
producer = KafkaProducer(bootstrap_servers='localhost:9092')

# Fazendo a requisição HTTP
response = requests.get(url)

# Verificando se a requisição foi bem-sucedida (código de status 200)
if response.status_code == 200:
    # Convertendo a resposta para JSON
    data = response.json()

    # Obtendo a lista de filmes
    filmes = data["results"]

    # Contador para controlar o número de filmes enviados
    filmes_enviados = 0

    for index, filme in enumerate(filmes):
        # Verificar se o título do filme está na lista de filmes desejados
        if filme['title'] in filmes_desejados:
            filme_url = f"https://api.themoviedb.org/3/movie/{filme['id']}?api_key=faa82585bb153b3362a84dc30fe51c35"

            print(f"Obtendo dados para o Filme {index + 1}/{len(filmes)}: {filme['title']}")

            filme_data = requests.get(filme_url).json()

            # Verifica se filme_data não está vazio
            if filme_data:
                # Estruturando os dados para envio ao Kafka
                record = {
                    "adult": filme_data.get("adult"),
                    "backdrop_path": filme_data.get("backdrop_path"),
                    "belongs_to_collection": filme_data.get("belongs_to_collection"),
                    "budget": filme_data.get("budget"),
                    "homepage": filme_data.get("homepage"),
                    "id": filme_data.get("id"),
                    "imdb_id": filme_data.get("imdb_id"),
                    "original_language": filme_data.get("original_language"),
                    "original_title": filme_data.get("original_title"),
                    "overview": filme_data.get("overview"),
                    "popularity": filme_data.get("popularity"),
                    "poster_path": filme_data.get("poster_path"),
                    "video": filme_data.get("video"),
                    "vote_average": filme_data.get("vote_average"),
                    "vote_count": filme_data.get("vote_count"),
                    "name": filme_data.get("title"),
                    "url": filme_url
                }

                # Convertendo os dados para formato JSON
                json_record = json.dumps(record)

                # Enviando o registro para o tópico do Kafka
                producer.send('atividade_kafka', value=json_record.encode('utf-8'))

                # Incrementar o contador de filmes enviados
                filmes_enviados += 1

                print(f"Dados do filme {filme['title']} enviados para o Kafka.")
            else:
                print(f"Erro: Dados vazios para o filme {filme['title']}. Ignorando...")
    
    producer.flush()
    producer.close()

    print(f"{filmes_enviados} filmes enviados com sucesso para o tópico do Kafka!")
else:
    print(f"Erro ao acessar a API de filmes da TMDb. Código de status: {response.status_code}")


Obtendo dados para o Filme 7/20: Muzzle
Dados do filme Muzzle enviados para o Kafka.
Obtendo dados para o Filme 8/20: Desperation Road
Dados do filme Desperation Road enviados para o Kafka.
Obtendo dados para o Filme 10/20: Coco
Dados do filme Coco enviados para o Kafka.
Obtendo dados para o Filme 12/20: Sound of Freedom
Dados do filme Sound of Freedom enviados para o Kafka.
Obtendo dados para o Filme 15/20: Blue Beetle
Dados do filme Blue Beetle enviados para o Kafka.
Obtendo dados para o Filme 16/20: The Burial
Dados do filme The Burial enviados para o Kafka.
Obtendo dados para o Filme 19/20: Scarygirl
Dados do filme Scarygirl enviados para o Kafka.
7 filmes enviados com sucesso para o tópico do Kafka!
